In [726]:
from scipy.optimize import linprog
import pandas as pd
import numpy as np

https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html

In [727]:
food_list = []
serving_list = []
price_list = []
calorie_list = []
carb_list = []
protein_list = []
fat_list = []

def add_food(name, serving, price, carbs, protein, fat):
    food_list.append(name)
    serving_list.append(serving)
    price_list.append(price)
    calorie_list.append(carbs*4 + protein*4 + fat*9)
    carb_list.append(carbs)
    protein_list.append(protein)
    fat_list.append(fat)

## ADD FOODS HERE

In [728]:
add_food("Quark (Magerstuffe Aldi)",100,0.60/5,4.1,12,0.2)
add_food("Berries (Aldi)",100,3/5,9.3,1.7,0.4)
add_food("Almonds (Aldi)",100,2.60/2,5.3,18,0.54)

## ===========

In [729]:
d = {'Food': food_list, 'Serving': serving_list,'Price':price_list,"Calories":calorie_list,"Carbs":carb_list,"Proteins":protein_list,"Fats":fat_list}
df = pd.DataFrame(data=d)

In [730]:
daily_calories = 469
daily_carbs_percentage = 14
daily_protein_percentage = 17
daily_fat_percentage = 69

daily_carbs = daily_carbs_percentage * daily_calories / 4
daily_protein = daily_protein_percentage * daily_calories / 4
daily_fat = daily_fat_percentage * daily_calories / 9

In [731]:
A_eq = []
b_eq = []
def add_equality_constrain(params, const):
    A_eq.append(params)
    b_eq.append(const)
    
A_ub = []
b_ub = []
def add_inequality_constrain(params, const):
    A_ub.append(params)
    b_ub.append(const)
    
bounds = []
for i in range(df.shape[0]):
    bounds.append((0,None))
    
def add_bounds(food_index, min_servings, max_servings):
    bounds[food_index] = (min_servings,max_servings)

In [732]:
df

,Food,Serving,Price,Calories,Carbs,Proteins,Fats
0,Quark (Magerstuffe Aldi),100,0.12,66.20,4.1,12.0,0.20
1,Berries (Aldi),100,0.60,47.60,9.3,1.7,0.40
2,Almonds (Aldi),100,1.30,98.06,5.3,18.0,0.54


## ADD CONSTRAINS HERE

In [733]:
c = list(df.Price)

#add_equality_constrain(list(df.Calories), daily_calories)
add_equality_constrain(list(df.Proteins), daily_protein)
add_equality_constrain(list(df.Carbs), daily_carbs)
add_equality_constrain(list(df.Fats), daily_fat)

#add_bounds(0, 0.5, None)
#add_bounds(1, 1, None)
#add_bounds(2, 0.5, None)

## ===============

In [734]:
if(len(A_eq) == 0):
    A_eq = None
    B_eq = None
    print("A_eq is empty")
    
if(len(A_ub) == 0):
    A_ub = None
    b_ub = None
    print("A_ub is empty")
    
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, method='revised simplex', options={'tol':1e+04})

A_ub is empty


ValueError: illegal value in 4th argument of internal getrf (lu_factor)

In [ ]:
if(res.success):
    food_ammount = round(res.x * df.Serving,4)
    res_calories = round(res.x * df.Calories,4)
    res_carbs = round(res.x * df.Carbs,4)
    res_proteins = round(res.x * df.Proteins,4)
    res_fats = round(res.x * df.Fats,4)
    
    out = {'Food':df.Food, "Servings":res.x,'Ammount g':food_ammount,'Carbs':res_carbs,'Protein':res_proteins,'Fat':res_fats,'Calories':res_calories}
    out_df = pd.DataFrame(data=out)
    print(out_df)
    
    print(100*"=")
    total_price = round(sum(res.x * df.Price),2)
    total_calories = round(sum(res.x * df.Calories),2)
    total_carbs = round(sum(res.x * df.Carbs),2)
    total_protein = round(sum(res.x * df.Proteins),2)
    total_fat = round(sum(res.x * df.Fats),2)
    
    print("Total price:",total_price,"Eur")
    print("Total calories:",total_calories,'g of',daily_calories, 'g ---> Delta is', daily_calories - total_calories, 'g')
    print("Total carbs:",total_carbs,'g of',daily_carbs, 'g ---> Delta is', daily_carbs - total_carbs, 'g')
    print("Total protein:",total_protein,'g of',daily_carbs, 'g ---> Delta is', daily_protein - total_protein, 'g')
    print("Total fat:",total_fat,'g of',daily_carbs, 'g ---> Delta is', daily_fat - total_fat, 'g')
else:
    print("There is no solution")
    print(res)